# 🎯 LIDAR Grasp Detection v14.1

## ✨ ใหม่ใน v14.1: Smart Descent with Fallback
| Feature | Description |
|---------|-------------|
| **⬇️ Descent Mode** | ลงจนกว่า LIDAR จะอ่านค่าใกล้ 0 |
| **⚠️ Gripper Limit** | ถ้า gripper ติด floor ก่อน → fallback |
| **📏 Fallback Mode** | ใช้วิธีวัดที่ Z=120 แทน (เหมือน v13) |

## 1️⃣ Imports

In [3]:
import cv2
import numpy as np
import time
import socket
import serial

print("✓ Imports (v14.1)")

✓ Imports (v14.1)


## 2️⃣ Hardware Configuration

In [4]:
ROBOT_IP = '192.168.1.6'
ESP32_PORT = 'COM9'
ESP32_BAUDRATE = 115200
CAMERA_ID = 2

HOMOGRAPHY_MATRIX = np.load('homography_matrix.npy')

print("✓ Hardware config")

✓ Hardware config


## 3️⃣ Configuration

In [5]:
# === CALIBRATED VALUES ===
PIXELS_PER_MM = 2.7703

# === Robot R Rotation ===
ROBOT_R_OFFSET = -25.55

# === Z Heights ===
Z_FLOOR = -64
Z_MEASURE = 195           # ความสูงเริ่มต้น
Z_FALLBACK_MEASURE = 120  # ความสูงสำหรับ fallback mode

# === v14.1: Smart Descent ===
GRIPPER_OPEN_OFFSET = 39  # mm (LIDAR ถึง gripper ตอนกางสุด)
LIDAR_STOP_DISTANCE = 20  # mm (หยุดเมื่อ LIDAR อ่านได้ค่านี้)
DESCENT_STEP = 10         # mm (ลงทีละ)
DESCENT_WAIT = 1     # วินาที

# v14.1: คำนวณ Z ต่ำสุดที่ LIDAR ลงได้ (ก่อน gripper ติดพื้น)
Z_LIDAR_MIN = Z_FLOOR + GRIPPER_OPEN_OFFSET  # LIDAR ลงได้ถึงนี่

# === LIDAR XY Offset ===
LIDAR_X_OFFSET = 25.08
LIDAR_Y_OFFSET = 20.71

# === v14.1: Fallback (เหมือน v13) ===
LIDAR_PHYSICAL_OFFSET = 60
LIDAR_CORRECTION = -21

# === Drop Position ===
DROP_POS = (169.71, 58.01, -17.07, 13.78)

# === Gripper ===
GRIPPER_MAX_WIDTH_MM = 54
GRIPPER_OPEN_MARGIN_MM = 5
GRIPPER_GRIP_MARGIN_MM = 5

# === Detection ===
MIN_OBJECT_AREA = 800
MAX_OBJECT_AREA = 50000

print("✓ Configuration v14.1")
print(f"   Descent: stop at LIDAR≤{LIDAR_STOP_DISTANCE}mm")
print(f"   Gripper limit: Z_LIDAR_MIN={Z_LIDAR_MIN} (if reached → fallback to Z={Z_FALLBACK_MEASURE})")

✓ Configuration v14.1
   Descent: stop at LIDAR≤20mm
   Gripper limit: Z_LIDAR_MIN=-25 (if reached → fallback to Z=120)


## 4️⃣ SmartGripperController Class

In [6]:
class SmartGripperController:
    CALIB_ANGLES = np.array([22, 30, 40, 50, 60, 70, 80, 90, 96])
    CALIB_WIDTHS = np.array([54.0, 52.0, 48.0, 40.0, 32.0, 23.0, 12.0, 3.0, 0.0])
    
    def __init__(self, port='COM9', baudrate=115200):
        self.port = port
        self.baudrate = baudrate
        self.serial = None
        self.target_width = None
        
    def connect(self):
        try:
            self.serial = serial.Serial(self.port, self.baudrate, timeout=2)
            time.sleep(2)
            self.serial.reset_input_buffer()
            print(f"✅ Gripper+LIDAR on {self.port}")
            return True
        except Exception as e:
            print(f"❌ {e}")
            return False
    
    def disconnect(self):
        if self.serial: 
            self.serial.close()
            self.serial = None
    
    def send_command(self, cmd):
        if self.serial and self.serial.is_open:
            self.serial.reset_input_buffer()
            self.serial.write((cmd + '\n').encode())
            time.sleep(0.3)
    
    def mm_to_angle(self, width_mm):
        width = max(0.0, min(54.0, width_mm))
        return int(round(np.interp(width, self.CALIB_WIDTHS[::-1], self.CALIB_ANGLES[::-1])))
    
    def open_fully(self):
        angle = 22
        print(f"🦾 Open FULLY ({angle}°)")
        self.send_command(f'G{angle}')
    
    def open_for_object(self, width_mm):
        self.target_width = width_mm
        open_w = min(54.0, width_mm + GRIPPER_OPEN_MARGIN_MM)
        angle = self.mm_to_angle(open_w)
        print(f"🦾 Open: {open_w:.1f}mm ({angle}°)")
        self.send_command(f'G{angle}')
    
    def grip_object(self, width_mm):
        grip_w = max(0.0, width_mm - GRIPPER_GRIP_MARGIN_MM)
        angle = self.mm_to_angle(grip_w)
        print(f"🦾 Grip: {grip_w:.1f}mm ({angle}°)")
        self.send_command(f'G{angle}')
    
    def release(self):
        open_w = min(54.0, (self.target_width or 30) + 10)
        self.send_command(f'G{self.mm_to_angle(open_w)}')
        self.target_width = None
    
    def read_lidar(self, samples=3):
        if not self.serial or not self.serial.is_open:
            return None
        
        readings = []
        for _ in range(samples):
            self.serial.reset_input_buffer()
            self.serial.write(b'L\n')
            
            start = time.time()
            while time.time() - start < 1.0:
                if self.serial.in_waiting > 0:
                    response = self.serial.readline().decode().strip()
                    if response.startswith("LIDAR:") and "ERR" not in response:
                        try:
                            dist = int(response.split(":")[1])
                            readings.append(dist)
                        except:
                            pass
                        break
            time.sleep(0.02)
        
        if readings:
            return int(np.median(readings))
        return None

print("✓ SmartGripperController")

✓ SmartGripperController


## 5️⃣ DobotControllerTCP Class

In [7]:
class DobotControllerTCP:
    def __init__(self, homography_matrix=None, r_offset=-25.55):
        self.dashboard_port = 29999
        self.sock = None
        self.H = homography_matrix
        self.r_offset = r_offset
        
    def connect(self, ip):
        try:
            self.sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            self.sock.settimeout(5)
            self.sock.connect((ip, self.dashboard_port))
            self.send_command("ClearError()")
            time.sleep(0.5)
            self.send_command("EnableRobot()")
            time.sleep(4)
            self.send_command("User(1)")
            self.send_command("Tool(1)")
            self.send_command("SpeedFactor(50)")
            print("✅ Robot connected!")
            return True
        except Exception as e:
            print(f"Error: {e}")
            return False

    def send_command(self, cmd):
        if self.sock:
            self.sock.send((cmd + "\n").encode("utf-8"))
            return self.sock.recv(1024).decode("utf-8")

    def home(self):
        print("🤖 HOME...")
        self.send_command("MovJ(-253.07, 115.17, -17.07, -62.78)")
        time.sleep(4)

    def move_to(self, x, y, z, r=0):
        cmd = f"MovJ({x},{y},{z},{r})"
        return self.send_command(cmd)
    
    def move_to_and_wait(self, x, y, z, r=0, wait=2):
        self.move_to(x, y, z, r)
        time.sleep(wait)
    
    def joint_move(self, j1=0, j2=0, j3=0, j4=0):
        cmd = f"JointMovJ({j1},{j2},{j3},{j4})"
        return self.send_command(cmd)
    
    def joint_move_and_wait(self, j1=0, j2=0, j3=0, j4=0, wait=3):
        self.joint_move(j1, j2, j3, j4)
        time.sleep(wait)

    def pixel_to_robot(self, u, v):
        if self.H is None: return None, None
        pt = np.array([u, v, 1], dtype=np.float32)
        res = np.dot(self.H, pt)
        return res[0]/res[2], res[1]/res[2]
    
    def camera_angle_to_robot_r(self, camera_angle):
        return self.r_offset - camera_angle

print("✓ DobotControllerTCP")

✓ DobotControllerTCP


## 6️⃣ ObjectDetectorV14 Class

In [8]:
class ObjectDetectorV14:
    def __init__(self, pixels_per_mm):
        self.ppm = pixels_per_mm
    
    def detect(self, frame):
        objects = []
        sat_objects = self._detect_by_saturation(frame)
        objects.extend(sat_objects)
        
        if len(objects) == 0:
            edge_objects = self._detect_by_edge(frame)
            objects.extend(edge_objects)
        
        objects = self._remove_duplicates(objects)
        return objects
    
    def _detect_by_saturation(self, frame):
        objects = []
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        h, s, v = cv2.split(hsv)
        
        _, sat_mask = cv2.threshold(s, 50, 255, cv2.THRESH_BINARY)
        _, dark_mask = cv2.threshold(v, 80, 255, cv2.THRESH_BINARY_INV)
        combined_mask = cv2.bitwise_or(sat_mask, dark_mask)
        
        kernel = np.ones((5, 5), np.uint8)
        combined_mask = cv2.morphologyEx(combined_mask, cv2.MORPH_OPEN, kernel)
        combined_mask = cv2.morphologyEx(combined_mask, cv2.MORPH_CLOSE, kernel)
        
        contours, _ = cv2.findContours(combined_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        for cnt in contours:
            area = cv2.contourArea(cnt)
            if MIN_OBJECT_AREA < area < MAX_OBJECT_AREA:
                obj = self._contour_to_object(cnt, 'color')
                if obj:
                    objects.append(obj)
        
        return objects
    
    def _detect_by_edge(self, frame):
        objects = []
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(gray, (5, 5), 0)
        edges = cv2.Canny(blur, 50, 150)
        
        kernel = np.ones((3, 3), np.uint8)
        edges = cv2.dilate(edges, kernel, iterations=2)
        edges = cv2.erode(edges, kernel, iterations=1)
        
        contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        for cnt in contours:
            area = cv2.contourArea(cnt)
            if MIN_OBJECT_AREA < area < MAX_OBJECT_AREA:
                obj = self._contour_to_object(cnt, 'edge')
                if obj:
                    objects.append(obj)
        
        return objects
    
    def _contour_to_object(self, cnt, method):
        hull = cv2.convexHull(cnt)
        rect = cv2.minAreaRect(hull)
        (cx, cy), (w, h), angle = rect
        x, y, bw, bh = cv2.boundingRect(cnt)
        
        return {
            'bbox': (x, y, bw, bh),
            'center': (int(cx), int(cy)),
            'rect': rect,
            'contour': hull,
            'area': cv2.contourArea(hull),
            'method': method
        }
    
    def _remove_duplicates(self, objects, min_dist=30):
        if len(objects) <= 1:
            return objects
        
        unique = []
        for obj in sorted(objects, key=lambda o: o['area'], reverse=True):
            cx, cy = obj['center']
            is_dup = False
            for u in unique:
                ux, uy = u['center']
                if np.sqrt((cx-ux)**2 + (cy-uy)**2) < min_dist:
                    is_dup = True
                    break
            if not is_dup:
                unique.append(obj)
        
        return unique

print("✓ ObjectDetectorV14")

✓ ObjectDetectorV14


## 7️⃣ PCAGraspSelector Class

In [9]:
class PCAGraspSelector:
    def __init__(self, pixels_per_mm):
        self.ppm = pixels_per_mm
    
    def analyze_object(self, obj):
        cnt = obj.get('contour')
        if cnt is None or len(cnt) < 5:
            return self._fallback(obj)
        
        pts = cnt.reshape(-1, 2).astype(np.float64)
        mean = np.mean(pts, axis=0)
        pts_centered = pts - mean
        
        cov = np.cov(pts_centered.T)
        eigenvalues, eigenvectors = np.linalg.eig(cov)
        idx = np.argsort(eigenvalues)[::-1]
        eigenvectors = eigenvectors[:, idx]
        
        major = eigenvectors[:, 0]
        minor = eigenvectors[:, 1]
        
        angle = np.degrees(np.arctan2(major[1], major[0]))
        
        proj = np.dot(pts_centered, minor)
        width_mm = (np.max(proj) - np.min(proj)) / self.ppm
        
        cx, cy = int(mean[0]), int(mean[1])
        grasp_angle = self._normalize(angle + 90)
        
        grasps = []
        if width_mm <= GRIPPER_MAX_WIDTH_MM:
            grasps.append({
                'center': (cx, cy),
                'width_mm': width_mm,
                'camera_angle': grasp_angle,
                'score': 1.0,
                'type': 'PCA'
            })
        return grasps if grasps else self._fallback(obj)
    
    def _fallback(self, obj):
        rect = obj.get('rect')
        if not rect: return []
        (cx, cy), (w, h), angle = rect
        grip_w = min(w, h) / self.ppm
        grip_a = angle + 90 if w < h else angle
        if grip_w <= GRIPPER_MAX_WIDTH_MM:
            return [{'center': (int(cx), int(cy)), 'width_mm': grip_w, 
                     'camera_angle': self._normalize(grip_a), 'score': 0.6, 'type': 'Rect'}]
        return []
    
    def _normalize(self, a):
        while a > 90: a -= 180
        while a < -90: a += 180
        return a

print("✓ PCAGraspSelector")

✓ PCAGraspSelector


## 8️⃣ Initialize Components

In [10]:
gripper = SmartGripperController(port=ESP32_PORT, baudrate=ESP32_BAUDRATE)
robot = DobotControllerTCP(homography_matrix=HOMOGRAPHY_MATRIX, r_offset=ROBOT_R_OFFSET)
detector = ObjectDetectorV14(PIXELS_PER_MM)
grasp_selector = PCAGraspSelector(PIXELS_PER_MM)
print("✓ Components initialized (v14.1)")

✓ Components initialized (v14.1)


---
# 🤖 CONNECT ROBOT & GRIPPER
---

In [11]:
print("="*60)
print("🤖 Connecting to Robot and Gripper+LIDAR...")
print("="*60)

gripper.connect()
robot.connect(ROBOT_IP)
robot.home()

print("✅ Ready!")

🤖 Connecting to Robot and Gripper+LIDAR...
✅ Gripper+LIDAR on COM9
✅ Robot connected!
🤖 HOME...
✅ Ready!


---
# 🎯 PICK v14.1: Smart Descent with Fallback
---

In [20]:
selected_object = None
selected_grasp = None
current_grasps = []
detected_objects = []

def mouse_callback(event, x, y, flags, param):
    global selected_object, selected_grasp, current_grasps
    if event == cv2.EVENT_LBUTTONDOWN:
        for obj in detected_objects:
            bx, by, bw, bh = obj['bbox']
            if bx <= x <= bx+bw and by <= y <= by+bh:
                selected_object = obj
                current_grasps = grasp_selector.analyze_object(obj)
                selected_grasp = current_grasps[0] if current_grasps else None
                if selected_grasp:
                    print(f"\n📦 Object: W={selected_grasp['width_mm']:.1f}mm")
                break

def draw_grasps(frame, grasps, selected):
    for g in grasps:
        cx, cy = g['center']
        angle = g['camera_angle']
        is_sel = (selected and g == selected)
        color = (0, 0, 255) if is_sel else (0, 255, 0)
        thick = 3 if is_sel else 2
        
        length = 40
        dx = int(length * np.cos(np.radians(angle)))
        dy = int(length * np.sin(np.radians(angle)))
        cv2.line(frame, (cx-dx, cy-dy), (cx+dx, cy+dy), color, thick)
        cv2.circle(frame, (cx, cy), 5, color, -1)

In [33]:
def pick_v14_1_smart_descent(obj, grasp):
    """v14.1 FIXED: Smart Descent with correct Z calculation"""
    cx, cy = grasp['center']
    grip_w = grasp['width_mm']
    camera_angle = grasp['camera_angle']
    
    robot_r = ROBOT_R_OFFSET
    gripper_x, gripper_y = robot.pixel_to_robot(cx, cy)
    lidar_x = gripper_x + LIDAR_X_OFFSET
    lidar_y = gripper_y + LIDAR_Y_OFFSET
    
    print(f"\n🤖 Pick v14.1 FIXED: W={grip_w:.1f}mm")
    print(f"   Gripper: ({gripper_x:.1f}, {gripper_y:.1f})")
    print(f"   LIDAR:   ({lidar_x:.1f}, {lidar_y:.1f})")
    
    # 1. Safe position
    print("🔄 Safe position...")
    robot.joint_move_and_wait(0, 0, 0, 0, 3)
    
    # 2. Open gripper FULLY
    gripper.open_fully()
    time.sleep(2)
    
    # 3. Move LIDAR above object
    print(f"📏 Moving LIDAR above object (Z={Z_MEASURE})...")
    robot.move_to_and_wait(lidar_x, lidar_y, Z_MEASURE, robot_r, 2)
    
    # 4. SMART DESCENT - ลงจน LIDAR ใกล้วัตถุ หรือ ติด floor
    print(f"⬇️ Starting smart descent...")
    print(f"   Stop when: LIDAR≤{LIDAR_STOP_DISTANCE}mm OR Z≤{Z_FLOOR}")
    
    current_z = Z_MEASURE
    last_lidar = None
    descent_success = False
    
    while current_z > Z_FLOOR:
        lidar_dist = gripper.read_lidar(samples=5)
        
        if lidar_dist is not None:
            last_lidar = lidar_dist
            print(f"   Z={current_z:.1f} → LIDAR={lidar_dist}mm")
            
            if lidar_dist <= LIDAR_STOP_DISTANCE:
                print(f"   ✅ LIDAR≤{LIDAR_STOP_DISTANCE}mm → DESCENT SUCCESS!")
                descent_success = True
                break
        else:
            print(f"   Z={current_z:.1f} → LIDAR=ERR")
        
        # Go down one step
        current_z = max(Z_FLOOR, current_z - DESCENT_STEP)
        robot.move_to_and_wait(lidar_x, lidar_y, current_z, robot_r, DESCENT_WAIT)
        
        if current_z <= Z_FLOOR:
            print(f"   ⚠️ Reached Z_FLOOR={Z_FLOOR}")
            break
    
    # 5. Calculate Z_GRASP
    if descent_success and last_lidar is not None:
        # *** FIX: LIDAR = 20mm หมายความว่าวัตถุอยู่ห่าง LIDAR 20mm ***
        # วัตถุอยู่ที่ Z = current_z - last_lidar
        # Gripper ต้องไปที่ระดับวัตถุ (หรือต่ำกว่านิดหน่อย)
        object_z = current_z - last_lidar
        #z_grasp = object_z - 5  # ลงไปต่ำกว่าผิววัตถุ 5mm เพื่อจับได้
        z_grasp = object_z + 25
        z_grasp = max(Z_FLOOR, z_grasp)
        print(f"📍 [DESCENT] Object at Z={object_z:.1f}, Z_GRASP={z_grasp:.1f}")
    else:
        # Fallback mode
        print(f"📏 [FALLBACK] Moving to Z={Z_FALLBACK_MEASURE} to measure...")
        robot.move_to_and_wait(lidar_x, lidar_y, Z_FALLBACK_MEASURE, robot_r, 2)
        time.sleep(1)
        
        lidar_dist = gripper.read_lidar(samples=100)
        if lidar_dist is None:
            print("❌ LIDAR read failed! Aborting.")
            robot.home()
            return
        
        print(f"📏 [FALLBACK] LIDAR at Z={Z_FALLBACK_MEASURE}: {lidar_dist}mm")
        
        # *** FIX: คำนวณตำแหน่งวัตถุ แล้วลงไปจับ ***
        object_z = Z_FALLBACK_MEASURE - lidar_dist
        #z_grasp = object_z - 5  # ลงไป 5mm ต่ำกว่าผิววัตถุ
        z_grasp = object_z + 25
        z_grasp = max(Z_FLOOR, z_grasp)
        print(f"📍 [FALLBACK] Object at Z={object_z:.1f}, Z_GRASP={z_grasp:.1f}")
    
    # 6. Move Gripper above object
    print("🎯 Moving Gripper above object...")
    robot.move_to_and_wait(gripper_x, gripper_y, Z_MEASURE, robot_r, 2)
    
    # 7. Rotate first
    final_r = robot.camera_angle_to_robot_r(camera_angle)
    print(f"🔄 Rotating to R={final_r:.1f}°...")
    robot.move_to_and_wait(gripper_x, gripper_y, Z_MEASURE, final_r, 2)
    
    # 8. Go down to grasp
    print(f"⬇️ Going down to Z={z_grasp:.1f}...")
    robot.move_to_and_wait(gripper_x, gripper_y, z_grasp, final_r, 2)
    
    # 9. Grip
    gripper.grip_object(grip_w - 8.5)
    time.sleep(4)
    
    # 10. Lift 20mm
    z_lift = z_grasp + 20
    print(f"⬆️ Lifting to Z={z_lift:.1f}...")
    robot.move_to_and_wait(gripper_x, gripper_y, z_lift, final_r, 2)
    
    # 11. Go to drop
    robot.move_to_and_wait(*DROP_POS[:3], DROP_POS[3], 3)
    
    # 12. Release
    gripper.release()
    time.sleep(2)
    
    # 13. Safe return
    drop_x, drop_y, drop_z, drop_r = DROP_POS
    robot.move_to_and_wait(drop_x, drop_y, 150, drop_r, 2)
    robot.joint_move_and_wait(0, 0, 0, 0, 3)
    robot.home()
    print("✅ Complete!")

In [ ]:
# Main loop
cap = cv2.VideoCapture(CAMERA_ID)
cv2.namedWindow('Pick v14.1')
cv2.setMouseCallback('Pick v14.1', mouse_callback)

print("="*60)
print("🎯 PICK v14.1 (Smart Descent with Fallback)")
print("Click=Select | SPACE=Execute | H=Home | C=Reconnect | Q=Quit")
print("="*60)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break
    
    detected_objects = detector.detect(frame)
    
    display = frame.copy()
    for obj in detected_objects:
        x, y, w, h = obj['bbox']
        is_sel = (selected_object and obj['center'] == selected_object['center'])
        color = (0, 0, 255) if is_sel else (0, 255, 0)
        cv2.rectangle(display, (x, y), (x+w, y+h), color, 2)
        cv2.putText(display, obj.get('method', ''), (x, y-5), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.4, color, 1)
    
    if selected_object and current_grasps:
        draw_grasps(display, current_grasps, selected_grasp)
    
    cv2.rectangle(display, (0, 0), (640, 35), (30, 30, 30), -1)
    cv2.putText(display, f"v14.1 | Obj:{len(detected_objects)} | SPACE | H | C | Q",
               (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255, 255, 255), 1)
    
    if selected_grasp:
        cv2.putText(display, f"[W={selected_grasp['width_mm']:.1f}mm - SPACE]",
                   (10, 470), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
    
    cv2.imshow('Pick v14.1', display)
    
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'): break
    elif key == ord('r'):
        selected_object = None
        selected_grasp = None
        current_grasps = []
    elif key == ord('h'):
        robot.home()
    elif key == ord('c'):
        print("\n🔄 Reconnecting...")
        try:
            gripper.disconnect()
        except:
            pass
        time.sleep(1)
        gripper.connect()
        try:
            robot.sock.close()
        except:
            pass
        time.sleep(1)
        robot.connect(ROBOT_IP)
        robot.home()
        selected_object = None
        selected_grasp = None
        current_grasps = []
        print("✅ Reconnected!")
    elif key == ord(' ') and selected_object and selected_grasp:
        pick_v14_1_smart_descent(selected_object, selected_grasp)
        selected_object = None
        selected_grasp = None
        current_grasps = []

cap.release()
cv2.destroyAllWindows()
gripper.disconnect()

🎯 PICK v14.1 (Smart Descent with Fallback)
Click=Select | SPACE=Execute | H=Home | C=Reconnect | Q=Quit

🔄 Reconnecting...
✅ Gripper+LIDAR on COM9
✅ Robot connected!
🤖 HOME...
✅ Reconnected!

📦 Object: W=32.9mm

🤖 Pick v14.1 FIXED: W=32.9mm
   Gripper: (6.5, 30.6)
   LIDAR:   (31.5, 51.3)
🔄 Safe position...
🦾 Open FULLY (22°)
📏 Moving LIDAR above object (Z=195)...
⬇️ Starting smart descent...
   Stop when: LIDAR≤20mm OR Z≤-64
   Z=195.0 → LIDAR=170mm
   Z=185.0 → LIDAR=160mm
   Z=175.0 → LIDAR=150mm
   Z=165.0 → LIDAR=140mm
   Z=155.0 → LIDAR=130mm
   Z=145.0 → LIDAR=120mm
   Z=135.0 → LIDAR=110mm
   Z=125.0 → LIDAR=110mm
   Z=115.0 → LIDAR=100mm
   Z=105.0 → LIDAR=90mm
   Z=95.0 → LIDAR=80mm
   Z=85.0 → LIDAR=60mm
   Z=75.0 → LIDAR=50mm
   Z=65.0 → LIDAR=40mm
   Z=55.0 → LIDAR=30mm
   Z=45.0 → LIDAR=20mm
   ✅ LIDAR≤20mm → DESCENT SUCCESS!
📍 [DESCENT] Object at Z=25.0, Z_GRASP=50.0
🎯 Moving Gripper above object...
🔄 Rotating to R=-25.4°...
⬇️ Going down to Z=50.0...
🦾 Grip: 19.4mm (73°